# List of Exposures with Hologram at USDF


- work with Weakly_2023_01
- use jupyter kernel LSST


- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab
- creation date : 2022/09/19
- last update : 2023/07/20



In [1]:
import sys
print(sys.executable)

/opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-6.0.0/bin/python3


In [2]:
print(sys.path)

['/sdf/data/rubin/user/dagoret/AuxTelComm/notebooks_usdf/butlertools', '/home/d/dagoret/repos/repos_w_2023_27/atmospec/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-6.0.0/Linux64/ctrl_mpexec/g88183d3282+88d805ed8c/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-6.0.0/Linux64/spectractor/g7bf34a7a6e/lib/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-6.0.0/Linux64/obs_lsst/gfd7e96728c+6f6aa72e7b/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-6.0.0/Linux64/pipe_tasks/gfb267207ab+bb26f3254a/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-6.0.0/Linux64/shapelet/gd4112516d0+5ce2cc5508/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-6.0.0/Linux64/meas_modelfit/g807e3ba568+b7d80ac9cf/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-6.0.0/Linux64/scarlet_extensions/g9d18589735+d445b4966c/lib/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-6.0.0/Linux64/proxmin/g33b4157f25/l

In [3]:
! eups list -s | grep LOCAL

atmospec              LOCAL:/home/d/dagoret/repos/repos_w_2023_27/atmospec 	setup
eups                  LOCAL:/opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-6.0.0/eups 	setup


In [4]:
! echo $IMAGE_DESCRIPTION
! eups list -s lsst_distrib

w_2023_27
   gdf42428520+4e9f6d16c8 	current w_2023_27 setup


In [5]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline
from matplotlib.colors import LogNorm

from mpl_toolkits.axes_grid1 import make_axes_locatable

import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.io import fits
from astropy.time import Time

In [6]:
from IPython.display import display, HTML

In [7]:
import lsst.afw.display as afwDisplay
afwDisplay.setDefaultBackend('matplotlib')

In [8]:
import lsst.daf.butler as dafButler

In [9]:
#repo = "/sdf/group/rubin/repo/main"
repo="/sdf/group/rubin/repo/oga/"
butler = dafButler.Butler(repo)
registry = butler.registry

In [10]:
collection='LATISS/raw/all'

## List of Exposures

In [11]:
df_exposure = pd.DataFrame(columns=['id', 'obs_id','day_obs', 'seq_num','time_start','time_end' ,'type', 'target','filter','zenith_angle','expos','ra','dec','skyangle','azimuth','zenith','science_program','jd','mjd'])

In [12]:
print(butler.registry.dimensions["exposure"].RecordClass.fields)

exposure: 
  instrument: str
  id: int
  physical_filter: str
  obs_id: str
  exposure_time: float
  dark_time: float
  observation_type: str
  observation_reason: str
  day_obs: int
  seq_num: int
  seq_start: int
  seq_end: int
  group_name: str
  group_id: int
  target_name: str
  science_program: str
  tracking_ra: float
  tracking_dec: float
  sky_angle: float
  azimuth: float
  zenith_angle: float
  has_simulated: bool
  timespan: lsst.daf.butler.Timespan


In [13]:
for count, info in enumerate(registry.queryDimensionRecords('exposure',where= "instrument='LATISS'")):
    
    
    
    df_exposure.loc[count] = [info.id, info.obs_id, info.day_obs, info.seq_num,pd.to_datetime(info.timespan.begin.to_string()),pd.to_datetime(info.timespan.end.to_string()) ,info.observation_type, info.target_name, info.physical_filter, info.zenith_angle, \
                             info.exposure_time,info.tracking_ra, info.tracking_dec, info.sky_angle,info.azimuth ,info.zenith_angle, info.science_program,
                             info.timespan.begin.jd,info.timespan.begin.mjd ]
    
    if count < 5:
        print("-----------------------------------------------------",count,"---------------------------------------------------------")
        print(info)
        print("\t id:                  ",info.id)
        print("\t day_obs:             ",info.day_obs)
        print("\t seq_num:             ",info.seq_num)
        print("\t type-of-observation: ",info.observation_type)
        print("\t target:              ",info.target_name)
        
        mjd = Time(info.timespan.begin.to_string()).mjd
        jd = Time(info.timespan.begin.to_string()).jd
        print(mjd,jd)
    

----------------------------------------------------- 0 ---------------------------------------------------------
exposure:
  instrument: 'LATISS'
  id: 2022083100004
  physical_filter: 'unknown~unknown'
  obs_id: 'AT_O_20220831_000004'
  exposure_time: 0.0
  dark_time: 0.0160482
  observation_type: 'bias'
  observation_reason: 'bias'
  day_obs: 20220831
  seq_num: 4
  seq_start: 4
  seq_end: 4
  group_name: '2022083100004'
  group_id: 2022083100004
  target_name: 'UNKNOWN'
  science_program: 'unknown'
  tracking_ra: None
  tracking_dec: None
  sky_angle: None
  azimuth: None
  zenith_angle: None
  has_simulated: False
  timespan: Timespan(begin=astropy.time.Time('2022-08-31 22:58:13.633984', scale='tai', format='iso'), end=astropy.time.Time('2022-08-31 22:58:13.650000', scale='tai', format='iso'))
	 id:                   2022083100004
	 day_obs:              20220831
	 seq_num:              4
	 type-of-observation:  bias
	 target:               UNKNOWN
59822.957102245186 2459823.45710

In [14]:
info.toDict()

{'instrument': 'LATISS',
 'id': 2023072000004,
 'physical_filter': 'SDSSg_65mm~blue300lpmm_qn1',
 'obs_id': 'AT_O_20230720_000004',
 'exposure_time': 2.0,
 'dark_time': 2.23934,
 'observation_type': 'engtest',
 'observation_reason': 'engtest',
 'day_obs': 20230720,
 'seq_num': 4,
 'seq_start': 4,
 'seq_end': 4,
 'group_name': '2023-07-20T20:22:43.454',
 'group_id': 2698141634540000,
 'target_name': 'DaytimeCheckout002',
 'science_program': 'unknown',
 'tracking_ra': 172.67235828637396,
 'tracking_dec': -20.11497100076907,
 'sky_angle': 100.01970747350711,
 'azimuth': 359.0767420155183,
 'zenith_angle': 9.99908676917424,
 'has_simulated': False,
 'timespan': Timespan(begin=astropy.time.Time('2023-07-20 20:22:45.503007', scale='tai', format='iso'), end=astropy.time.Time('2023-07-20 20:22:47.742000', scale='tai', format='iso'))}

In [15]:
info

exposure.RecordClass(instrument='LATISS', id=2023072000004, physical_filter='SDSSg_65mm~blue300lpmm_qn1', obs_id='AT_O_20230720_000004', exposure_time=2.0, dark_time=2.23934, observation_type='engtest', observation_reason='engtest', day_obs=20230720, seq_num=4, seq_start=4, seq_end=4, group_name='2023-07-20T20:22:43.454', group_id=2698141634540000, target_name='DaytimeCheckout002', science_program='unknown', tracking_ra=172.67235828637396, tracking_dec=-20.11497100076907, sky_angle=100.01970747350711, azimuth=359.0767420155183, zenith_angle=9.99908676917424, has_simulated=False, timespan=Timespan(begin=astropy.time.Time('2023-07-20 20:22:45.503007', scale='tai', format='iso'), end=astropy.time.Time('2023-07-20 20:22:47.742000', scale='tai', format='iso')))

In [16]:
info.azimuth

359.0767420155183

In [17]:
df_exposure

,id,obs_id,day_obs,seq_num,time_start,time_end,type,target,filter,zenith_angle,expos,ra,dec,skyangle,azimuth,zenith,science_program,jd,mjd
0,2022083100004,AT_O_20220831_000004,20220831,4,2022-08-31 22:58:13.633984,2022-08-31 22:58:13.650,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN,NaN,NaN,unknown,2.459823e+06,59822.957102
1,2022083100008,AT_O_20220831_000008,20220831,8,2022-08-31 22:59:16.387013,2022-08-31 22:59:16.398,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN,NaN,NaN,unknown,2.459823e+06,59822.957829
2,2022083100007,AT_O_20220831_000007,20220831,7,2022-08-31 22:59:11.978990,2022-08-31 22:59:11.991,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN,NaN,NaN,unknown,2.459823e+06,59822.957778
3,2022083100009,AT_O_20220831_000009,20220831,9,2022-08-31 22:59:20.801996,2022-08-31 22:59:20.813,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN,NaN,NaN,unknown,2.459823e+06,59822.957880
4,2022083100012,AT_O_20220831_000012,20220831,12,2022-08-31 23:00:00.113994,2022-08-31 23:00:03.124,dark,UNKNOWN,unknown~unknown,NaN,3.0,NaN,NaN,NaN,NaN,NaN,unknown,2.459823e+06,59822.958335
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51486,2023071900120,AT_O_20230719_000120,20230719,120,2023-07-19 22:16:48.678000,2023-07-19 22:16:54.907,flat,FlatField position,SDSSg_65mm~empty,50.982214,6.0,191.170447,-80.983945,NaN,181.805648,50.982214,unknown,2.460145e+06,60144.928341
51487,2023072000001,AT_O_20230720_000001,20230720,1,2023-07-20 20:13:25.568004,2023-07-20 20:13:25.584,bias,Park position,SDSSg_65mm~empty,9.981405,0.0,262.904199,-20.243705,NaN,0.580577,9.981405,unknown,2.460146e+06,60145.842657
51488,2023072000002,AT_O_20230720_000002,20230720,2,2023-07-20 20:13:34.343011,2023-07-20 20:13:36.587,engtest,Park position,empty~empty,10.004850,2.0,170.509047,-20.108880,100.270350,359.838755,10.004850,unknown,2.460146e+06,60145.842759
51489,2023072000003,AT_O_20230720_000003,20230720,3,2023-07-20 20:19:52.881010,2023-07-20 20:19:55.110,engtest,DaytimeCheckout001,empty~empty,44.880503,2.0,202.033699,4.465883,172.768132,44.803184,44.880503,unknown,2.460146e+06,60145.847140


In [18]:
df_exposure = df_exposure.astype({"id": int,'day_obs': int,'seq_num':int})

In [19]:
df_exposure

,id,obs_id,day_obs,seq_num,time_start,time_end,type,target,filter,zenith_angle,expos,ra,dec,skyangle,azimuth,zenith,science_program,jd,mjd
0,2022083100004,AT_O_20220831_000004,20220831,4,2022-08-31 22:58:13.633984,2022-08-31 22:58:13.650,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN,NaN,NaN,unknown,2.459823e+06,59822.957102
1,2022083100008,AT_O_20220831_000008,20220831,8,2022-08-31 22:59:16.387013,2022-08-31 22:59:16.398,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN,NaN,NaN,unknown,2.459823e+06,59822.957829
2,2022083100007,AT_O_20220831_000007,20220831,7,2022-08-31 22:59:11.978990,2022-08-31 22:59:11.991,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN,NaN,NaN,unknown,2.459823e+06,59822.957778
3,2022083100009,AT_O_20220831_000009,20220831,9,2022-08-31 22:59:20.801996,2022-08-31 22:59:20.813,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN,NaN,NaN,unknown,2.459823e+06,59822.957880
4,2022083100012,AT_O_20220831_000012,20220831,12,2022-08-31 23:00:00.113994,2022-08-31 23:00:03.124,dark,UNKNOWN,unknown~unknown,NaN,3.0,NaN,NaN,NaN,NaN,NaN,unknown,2.459823e+06,59822.958335
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51486,2023071900120,AT_O_20230719_000120,20230719,120,2023-07-19 22:16:48.678000,2023-07-19 22:16:54.907,flat,FlatField position,SDSSg_65mm~empty,50.982214,6.0,191.170447,-80.983945,NaN,181.805648,50.982214,unknown,2.460145e+06,60144.928341
51487,2023072000001,AT_O_20230720_000001,20230720,1,2023-07-20 20:13:25.568004,2023-07-20 20:13:25.584,bias,Park position,SDSSg_65mm~empty,9.981405,0.0,262.904199,-20.243705,NaN,0.580577,9.981405,unknown,2.460146e+06,60145.842657
51488,2023072000002,AT_O_20230720_000002,20230720,2,2023-07-20 20:13:34.343011,2023-07-20 20:13:36.587,engtest,Park position,empty~empty,10.004850,2.0,170.509047,-20.108880,100.270350,359.838755,10.004850,unknown,2.460146e+06,60145.842759
51489,2023072000003,AT_O_20230720_000003,20230720,3,2023-07-20 20:19:52.881010,2023-07-20 20:19:55.110,engtest,DaytimeCheckout001,empty~empty,44.880503,2.0,202.033699,4.465883,172.768132,44.803184,44.880503,unknown,2.460146e+06,60145.847140


## Selection of science

In [20]:
df_science = df_exposure[df_exposure.type == 'science']

In [21]:
df_science.sort_values(by="id",ascending=True,inplace=True)

/tmp/ipykernel_20207/3986782663.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_science.sort_values(by="id",ascending=True,inplace=True)


In [22]:
df_science.tail(60)

,id,obs_id,day_obs,seq_num,time_start,time_end,type,target,filter,zenith_angle,expos,ra,dec,skyangle,azimuth,zenith,science_program,jd,mjd
51250,2023071800267,AT_O_20230718_000267,20230718,267,2023-07-19 00:38:55.292986,2023-07-19 00:39:25.532,science,Photo1600-1_007,SDSSi_65mm~empty,7.574884,30.0,240.039389,-24.048276,0.000102,37.210257,7.574884,AUXTEL_PHOTO_IMAGING,2.460145e+06,60144.027029
51251,2023071800268,AT_O_20230718_000268,20230718,268,2023-07-19 00:39:39.637988,2023-07-19 00:40:09.875,science,Photo1600-1_007,SDSSg_65mm~empty,7.480142,30.0,240.039460,-24.048186,0.000159,36.154037,7.480142,AUXTEL_PHOTO_IMAGING,2.460145e+06,60144.027542
51252,2023071800269,AT_O_20230718_000269,20230718,269,2023-07-19 00:40:23.948993,2023-07-19 00:40:54.188,science,Photo1600-1_007,SDSSr_65mm~empty,7.387707,30.0,240.039503,-24.048180,0.000610,35.072367,7.387707,AUXTEL_PHOTO_IMAGING,2.460145e+06,60144.028055
51253,2023071800270,AT_O_20230718_000270,20230718,270,2023-07-19 00:41:09.987005,2023-07-19 00:41:40.231,science,Photo1600-1_027,SDSSr_65mm~empty,7.338247,30.0,239.974757,-23.959410,0.000288,33.152533,7.338247,AUXTEL_PHOTO_IMAGING,2.460145e+06,60144.028588
51254,2023071800271,AT_O_20230718_000271,20230718,271,2023-07-19 00:41:54.382982,2023-07-19 00:42:24.623,science,Photo1600-1_027,SDSSg_65mm~empty,7.252577,30.0,239.974662,-23.959544,0.000539,32.014832,7.252577,AUXTEL_PHOTO_IMAGING,2.460145e+06,60144.029102
51255,2023071800272,AT_O_20230718_000272,20230718,272,2023-07-19 00:42:38.749992,2023-07-19 00:43:08.995,science,Photo1600-1_027,SDSSi_65mm~empty,7.168016,30.0,239.974690,-23.959447,0.000473,30.824545,7.168016,AUXTEL_PHOTO_IMAGING,2.460145e+06,60144.029615
51256,2023071800273,AT_O_20230718_000273,20230718,273,2023-07-19 00:45:58.934020,2023-07-19 00:46:29.163,science,Photo1800-1_011,SDSSi_65mm~empty,29.924899,30.0,270.174550,-25.018621,0.001279,88.195103,29.924899,AUXTEL_PHOTO_IMAGING,2.460145e+06,60144.031932
51257,2023071800274,AT_O_20230718_000274,20230718,274,2023-07-19 00:46:43.287993,2023-07-19 00:47:13.532,science,Photo1800-1_011,SDSSg_65mm~empty,29.769424,30.0,270.174506,-25.018607,0.001085,88.095576,29.769424,AUXTEL_PHOTO_IMAGING,2.460145e+06,60144.032445
51258,2023071800275,AT_O_20230718_000275,20230718,275,2023-07-19 00:47:27.650014,2023-07-19 00:47:57.883,science,Photo1800-1_011,SDSSr_65mm~empty,29.603417,30.0,270.174491,-25.018603,0.001164,87.988737,29.603417,AUXTEL_PHOTO_IMAGING,2.460145e+06,60144.032959
51259,2023071800276,AT_O_20230718_000276,20230718,276,2023-07-19 00:50:41.990018,2023-07-19 00:51:12.233,science,Photo1600-1_029,SDSSr_65mm~empty,6.472191,30.0,240.007067,-23.959509,359.999703,16.651281,6.472191,AUXTEL_PHOTO_IMAGING,2.460145e+06,60144.035208


In [23]:
df_science.day_obs.unique()

array([20220912, 20220913, 20220914, 20220927, 20220928, 20220929,
       20221011, 20221012, 20221013, 20221025, 20221026, 20221027,
       20221108, 20221109, 20221110, 20221122, 20221123, 20221124,
       20221207, 20221208, 20221209, 20221210, 20221212, 20230117,
       20230118, 20230119, 20230131, 20230201, 20230202, 20230214,
       20230215, 20230216, 20230228, 20230301, 20230302, 20230314,
       20230315, 20230316, 20230509, 20230510, 20230511, 20230523,
       20230524, 20230525, 20230704, 20230705, 20230706, 20230718])

## Selection of date

2021-02-16  2021-02-18  2021-06-08  2021-06-10  2021-07-07  2021-09-07  2021-09-09  2021-10-06  2021-11-02  2021-11-04  2022-02-16  2022-03-15  2022-03-17  20220502  
2021-02-17  2021-03-11  2021-06-09  2021-07-06  2021-07-08  2021-09-08  2021-10-05  2021-10-07  2021-11-03  2022-02-15  2022-02-17  2022-03-16  2022-03-18  20220503 20220524 20220607 20220608 20220609

### Add those

20220504 - Not existing

In [24]:
#date_sel = 20220630
#date_sel = 20220630
#date_sel = 20221208
#date_sel = 20221209

# data in january
#date_sel= 20230118
date_sel= 20230131

# data in February
#date_sel = 20230201
#date_sel = 20230202
#date_sel = 20230704

#date_sel = 20230510
#date_sel = 20230509
#date_sel = 20230511

#date_sel = 20230705
#date_sel = 20230706

#date_sel = 20230718


## Query Dimension records on Exposures

In [26]:
where_clause = f"instrument=\'LATISS\' AND exposure.day_obs={date_sel} AND exposure.science_program=\'spec\'"

In [27]:
results_exposures = registry.queryDimensionRecords('exposure',datasets="raw",collections = 'LATISS/raw/all',where=where_clause)

## Select date

In [28]:
df_science_selected = df_science[df_science.day_obs == date_sel ]

In [29]:
len(df_science_selected)

339

# List of filters

In [30]:
list_of_filters = df_science_selected['filter'].unique()
list_of_filters

array(['empty~holo4_003', 'BG40_65mm_1~holo4_003',
       'OG550_65mm_1~holo4_003', 'SDSSr_65mm~empty'], dtype=object)

In [31]:
selected_filters = []
for filt in list_of_filters:
    #flag_sel = (filt.find('holo4') != -1) or (filt.find('ronchi90lpmm') != -1) or (filt.find('ronchi170lpmm') != -1)
    flag_sel = (filt.find('holo4') != -1) 
    #flag_sel = (filt.find('ronchi170') != -1) 
    if flag_sel:
        selected_filters.append(filt) 
selected_filters = np.array(selected_filters)
selected_filters       

array(['empty~holo4_003', 'BG40_65mm_1~holo4_003',
       'OG550_65mm_1~holo4_003'], dtype='<U22')

## Selection of filter

- But now we have a loop on filters

In [32]:
#idx_sel_filt = 0
#selected_filter = selected_filters[0]
#selected_filter

## Final selection of exposure

- loop on selected filters

In [33]:
pd.set_option('display.max_rows', None)

In [34]:
all_def_science_selected_final = []

for selected_filter in selected_filters:

    print(selected_filter)
    df_science_selected_final = df_science_selected[df_science_selected["filter"] ==  selected_filter]
    
    #print(df_science_selected_final)
    all_def_science_selected_final.append(df_science_selected_final.copy())
    
    df = df_science_selected_final
    df_sel = df[['day_obs', 'seq_num']]
    
    
    NN = len(df)
    
    if NN>0:
        # build the line
        
        df_sel['line'] = df_sel.apply(lambda row: str(row['day_obs'])  + " " + str(row['seq_num']),axis = 1)
    
    
         # prepare output
        filename_out = "visitdispersers_"+ str(date_sel) + "_filt_" + selected_filter.replace('~','-') + ".list"
        print(filename_out)
        
        df_out = df_sel[["line"]]
        df_out.to_csv(filename_out, index=False, header=None)

empty~holo4_003
visitdispersers_20230131_filt_empty-holo4_003.list
BG40_65mm_1~holo4_003
visitdispersers_20230131_filt_BG40_65mm_1-holo4_003.list
OG550_65mm_1~holo4_003
visitdispersers_20230131_filt_OG550_65mm_1-holo4_003.list


/tmp/ipykernel_20207/3368292521.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sel['line'] = df_sel.apply(lambda row: str(row['day_obs'])  + " " + str(row['seq_num']),axis = 1)
/tmp/ipykernel_20207/3368292521.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sel['line'] = df_sel.apply(lambda row: str(row['day_obs'])  + " " + str(row['seq_num']),axis = 1)
/tmp/ipykernel_20207/3368292521.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using

In [35]:
N = len(all_def_science_selected_final)

In [36]:
if N>=1:
    n0=len(all_def_science_selected_final[0])
    display(all_def_science_selected_final[0])

,id,obs_id,day_obs,seq_num,time_start,time_end,type,target,filter,zenith_angle,expos,ra,dec,skyangle,azimuth,zenith,science_program,jd,mjd
33344,2023013100219,AT_O_20230131_000219,20230131,219,2023-02-01 00:57:27.188987,2023-02-01 00:57:57.418,science,HD185975,empty~holo4_003,61.206717,30.0,305.756766,-87.535126,217.941160,182.329803,61.206717,spec,2.459977e+06,59976.039898
33314,2023013100220,AT_O_20230131_000220,20230131,220,2023-02-01 00:58:10.015993,2023-02-01 00:58:40.247,science,HD185975,empty~holo4_003,61.213193,30.0,305.756496,-87.535117,217.941313,182.324292,61.213193,spec,2.459977e+06,59976.040394
33318,2023013100228,AT_O_20230131_000228,20230131,228,2023-02-01 01:03:22.186981,2023-02-01 01:03:52.425,science,HD14943,empty~holo4_003,36.034382,30.0,35.760158,-51.174951,160.089833,223.224364,36.034382,spec,2.459977e+06,59976.044007
33349,2023013100229,AT_O_20230131_000229,20230131,229,2023-02-01 01:04:05.005980,2023-02-01 01:04:35.234,science,HD14943,empty~holo4_003,36.140845,30.0,35.760076,-51.174947,160.089958,223.288435,36.140845,spec,2.459977e+06,59976.044502
33353,2023013100237,AT_O_20230131_000237,20230131,237,2023-02-01 01:08:46.860014,2023-02-01 01:09:17.098,science,HD14943,empty~holo4_003,36.841276,30.0,35.756593,-51.175835,161.663133,223.688468,36.841276,spec,2.459977e+06,59976.047765
33323,2023013100238,AT_O_20230131_000238,20230131,238,2023-02-01 01:09:29.678007,2023-02-01 01:09:59.909,science,HD14943,empty~holo4_003,36.948514,30.0,35.756709,-51.175784,161.663013,223.746917,36.948514,spec,2.459977e+06,59976.048260
33358,2023013100247,AT_O_20230131_000247,20230131,247,2023-02-01 01:20:52.298002,2023-02-01 01:21:22.537,science,HD38949,empty~holo4_003,8.435964,30.0,87.133078,-24.538619,137.580536,49.326944,8.435964,spec,2.459977e+06,59976.056161
33328,2023013100248,AT_O_20230131_000248,20230131,248,2023-02-01 01:21:35.297004,2023-02-01 01:22:05.539,science,HD38949,empty~holo4_003,8.319188,30.0,87.133117,-24.538552,137.580272,48.549816,8.319188,spec,2.459977e+06,59976.056659
33364,2023013100259,AT_O_20230131_000259,20230131,259,2023-02-01 01:30:57.858990,2023-02-01 01:31:28.094,science,HD185975,empty~holo4_003,61.496498,30.0,305.615698,-87.526586,226.199181,182.049675,61.496498,spec,2.459977e+06,59976.063170
33334,2023013100260,AT_O_20230131_000260,20230131,260,2023-02-01 01:31:40.855981,2023-02-01 01:32:11.095,science,HD185975,empty~holo4_003,61.502199,30.0,305.615627,-87.526590,226.199145,182.043159,61.502199,spec,2.459977e+06,59976.063667


In [37]:
if N>=2:
    n1=len(all_def_science_selected_final[1])
    display(all_def_science_selected_final[1])

,id,obs_id,day_obs,seq_num,time_start,time_end,type,target,filter,zenith_angle,expos,ra,dec,skyangle,azimuth,zenith,science_program,jd,mjd
33345,2023013100221,AT_O_20230131_000221,20230131,221,2023-02-01 00:58:54.489982,2023-02-01 00:59:24.721,science,HD185975,BG40_65mm_1~holo4_003,61.219786,30.0,305.756421,-87.535125,217.941266,182.318622,61.219786,spec,2.459977e+06,59976.040908
33315,2023013100222,AT_O_20230131_000222,20230131,222,2023-02-01 00:59:37.304998,2023-02-01 01:00:07.538,science,HD185975,BG40_65mm_1~holo4_003,61.226216,30.0,305.756516,-87.535121,217.941134,182.313072,61.226216,spec,2.459977e+06,59976.041404
33319,2023013100230,AT_O_20230131_000230,20230131,230,2023-02-01 01:04:49.481016,2023-02-01 01:05:19.721,science,HD14943,BG40_65mm_1~holo4_003,36.249839,30.0,35.760133,-51.174965,160.089712,223.353122,36.249839,spec,2.459977e+06,59976.045017
33350,2023013100231,AT_O_20230131_000231,20230131,231,2023-02-01 01:05:32.301986,2023-02-01 01:06:02.537,science,HD14943,BG40_65mm_1~holo4_003,36.356505,30.0,35.760136,-51.174964,160.089772,223.415622,36.356505,spec,2.459977e+06,59976.045513
33354,2023013100239,AT_O_20230131_000239,20230131,239,2023-02-01 01:10:14.148013,2023-02-01 01:10:44.374,science,HD14943,BG40_65mm_1~holo4_003,37.061003,30.0,35.756636,-51.175813,161.663054,223.807243,37.061003,spec,2.459977e+06,59976.048775
33324,2023013100240,AT_O_20230131_000240,20230131,240,2023-02-01 01:10:57.051019,2023-02-01 01:11:27.292,science,HD14943,BG40_65mm_1~holo4_003,37.168494,30.0,35.756734,-51.175799,161.662752,223.864163,37.168494,spec,2.459977e+06,59976.049271
33359,2023013100249,AT_O_20230131_000249,20230131,249,2023-02-01 01:22:19.883003,2023-02-01 01:22:50.113,science,HD38949,BG40_65mm_1~holo4_003,8.201009,30.0,87.133045,-24.538605,137.580133,47.733294,8.201009,spec,2.459977e+06,59976.057175
33329,2023013100250,AT_O_20230131_000250,20230131,250,2023-02-01 01:23:02.684983,2023-02-01 01:23:32.925,science,HD38949,BG40_65mm_1~holo4_003,8.087079,30.0,87.133034,-24.538597,137.559723,46.913728,8.087079,spec,2.459977e+06,59976.057670
33365,2023013100261,AT_O_20230131_000261,20230131,261,2023-02-01 01:32:25.461009,2023-02-01 01:32:55.700,science,HD185975,BG40_65mm_1~holo4_003,61.508163,30.0,305.615290,-87.526584,226.199322,182.036318,61.508163,spec,2.459977e+06,59976.064184
33335,2023013100262,AT_O_20230131_000262,20230131,262,2023-02-01 01:33:08.350014,2023-02-01 01:33:38.583,science,HD185975,BG40_65mm_1~holo4_003,61.513821,30.0,305.615320,-87.526594,226.199161,182.029762,61.513821,spec,2.459977e+06,59976.064680


In [38]:
if N>=3:
    n2=len(all_def_science_selected_final[2])
    display(all_def_science_selected_final[2])

,id,obs_id,day_obs,seq_num,time_start,time_end,type,target,filter,zenith_angle,expos,ra,dec,skyangle,azimuth,zenith,science_program,jd,mjd
33346,2023013100223,AT_O_20230131_000223,20230131,223,2023-02-01 01:00:22.889985,2023-02-01 01:00:53.116,science,HD185975,OG550_65mm_1~holo4_003,61.233081,30.0,305.756328,-87.535127,217.941097,182.307094,61.233081,spec,2.459977e+06,59976.041932
33316,2023013100224,AT_O_20230131_000224,20230131,224,2023-02-01 01:01:05.696995,2023-02-01 01:01:35.927,science,HD185975,OG550_65mm_1~holo4_003,61.239340,30.0,305.756195,-87.535122,217.941061,182.301623,61.239340,spec,2.459977e+06,59976.042427
33320,2023013100232,AT_O_20230131_000232,20230131,232,2023-02-01 01:06:18.502017,2023-02-01 01:06:48.732,science,HD14943,OG550_65mm_1~holo4_003,36.470765,30.0,35.760112,-51.174963,160.089777,223.481657,36.470765,spec,2.459977e+06,59976.046047
33351,2023013100233,AT_O_20230131_000233,20230131,233,2023-02-01 01:07:01.308987,2023-02-01 01:07:31.542,science,HD14943,OG550_65mm_1~holo4_003,36.577666,30.0,35.760142,-51.174959,160.089642,223.542601,36.577666,spec,2.459977e+06,59976.046543
33355,2023013100241,AT_O_20230131_000241,20230131,241,2023-02-01 01:11:42.753004,2023-02-01 01:12:12.981,science,HD14943,OG550_65mm_1~holo4_003,37.281238,30.0,35.756626,-51.175815,161.662984,223.922986,37.281238,spec,2.459977e+06,59976.049800
33325,2023013100242,AT_O_20230131_000242,20230131,242,2023-02-01 01:12:25.560014,2023-02-01 01:12:55.800,science,HD14943,OG550_65mm_1~holo4_003,37.389007,30.0,35.756648,-51.175826,161.662891,223.978437,37.389007,spec,2.459977e+06,59976.050296
33360,2023013100251,AT_O_20230131_000251,20230131,251,2023-02-01 01:23:48.499983,2023-02-01 01:24:18.729,science,HD38949,OG550_65mm_1~holo4_003,7.966976,30.0,87.133075,-24.538569,137.559103,46.012855,7.966976,spec,2.459977e+06,59976.058200
33330,2023013100252,AT_O_20230131_000252,20230131,252,2023-02-01 01:24:31.309005,2023-02-01 01:25:01.547,science,HD38949,OG550_65mm_1~holo4_003,7.856293,30.0,87.132994,-24.538557,137.558091,45.146873,7.856293,spec,2.459977e+06,59976.058696
33366,2023013100263,AT_O_20230131_000263,20230131,263,2023-02-01 01:33:54.056988,2023-02-01 01:34:24.300,science,HD185975,OG550_65mm_1~holo4_003,61.519875,30.0,305.615039,-87.526588,226.199307,182.022727,61.519875,spec,2.459977e+06,59976.065209
33336,2023013100264,AT_O_20230131_000264,20230131,264,2023-02-01 01:34:36.881017,2023-02-01 01:35:07.123,science,HD185975,OG550_65mm_1~holo4_003,61.525368,30.0,305.615224,-87.526588,226.199032,182.016297,61.525368,spec,2.459977e+06,59976.065705


In [39]:
if N>=4:
    n3=len(all_def_science_selected_final[3])
    display(all_def_science_selected_final[3])

In [40]:
if N>=5:
    n4=len(all_def_science_selected_final[4])
    display(all_def_science_selected_final[4])

In [41]:
df_forsave = pd.concat(all_def_science_selected_final)

In [42]:
filename_out_csv = "holosummary_"+ str(date_sel) + ".csv"
filename_out_excel = "holosummary_"+ str(date_sel) + ".xlsx"

In [43]:
filename_out_csv

'holosummary_20230131.csv'

In [44]:
df_forsave.to_csv(filename_out_csv) 

In [45]:
! pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable


In [46]:
df_forsave.to_excel(filename_out_excel) 